<a href="https://colab.research.google.com/github/Abintilas/CCTv-GunDetection/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/weaponsdata/

/content/drive/MyDrive/weaponsdata


In [4]:
%pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 709.3/709.3 kB 9.8 MB/s eta 0:00:00


In [5]:
from ultralytics import YOLO
model = YOLO("yolov8n.pt")

In [8]:
#!python detect.py --weights yolov8s.pt --img 640 --source datasets/images
!yolo detect train data='/content/drive/MyDrive/weaponsdata/data.yaml' model='yolov8n.pt' epochs=1 imgsz=640 save=true

New https://pypi.org/project/ultralytics/8.1.10 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.9 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/weaponsdata/data.yaml, epochs=1, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train10, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, em

In [9]:
!yolo detect val data='/content/drive/MyDrive/weaponsdata/data.yaml' model='/content/drive/MyDrive/weaponsdata/runs/detect/train10/weights/best.pt'

Ultralytics YOLOv8.1.9 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning /content/drive/MyDrive/weaponsdata/valid/labels.cache... 1665 images, 51 backgrounds, 0 corrupt: 100% 1675/1675 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 105/105 [00:41<00:00,  2.54it/s]
                   all       1675       3163      0.531      0.397       0.41      0.144
                person       1675       1420      0.694      0.482      0.545      0.175
                weapon       1675       1743      0.368      0.312      0.274      0.114
Speed: 0.4ms preprocess, 2.6ms inference, 0.0ms loss, 2.7ms postprocess per image
Results saved to runs/detect/val
💡 Learn more at https://docs.ultralytics.com/modes/val


In [10]:
!yolo detect val data='/content/drive/MyDrive/weaponsdata/data.yaml' model='/content/drive/MyDrive/weaponsdata/runs/detect/train10/weights/last.pt'

Ultralytics YOLOv8.1.9 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning /content/drive/MyDrive/weaponsdata/valid/labels.cache... 1665 images, 51 backgrounds, 0 corrupt: 100% 1675/1675 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 105/105 [00:49<00:00,  2.12it/s]
                   all       1675       3163      0.531      0.397       0.41      0.144
                person       1675       1420      0.694      0.482      0.545      0.175
                weapon       1675       1743      0.368      0.312      0.274      0.114
Speed: 0.6ms preprocess, 3.1ms inference, 0.0ms loss, 3.6ms postprocess per image
Results saved to runs/detect/val2
💡 Learn more at https://docs.ultralytics.com/modes/val


In [33]:
import torch
import torchvision
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F
from PIL import Image
import cv2
from google.colab.patches import cv2_imshow

WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.


In [34]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
  ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.
WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.
WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.


In [40]:
image_path = take_photo()

<IPython.core.display.Javascript object>

WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.
WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.
WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.
WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.
WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.
WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.
WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.
WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.
WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.
WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.
WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.
WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.
WARNING ⚠️ Video stream unresponsive, please check your IP camer

In [41]:
model = YOLO('/content/drive/MyDrive/weaponsdata/runs/detect/train10/weights/best.pt')
image = Image.open(image_path)

results = model(image)  # list of Results objects

WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.
WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.
WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.
WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.
WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.
WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.
WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.
WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.
WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.

WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.
WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.
WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.
0: 480x640 3 persons, 25.1ms
Speed: 1.6ms preprocess, 25.1ms in

In [42]:
model.predict((image), save=True, imgsz=320, conf=0.5)


WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.
WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.
0: 256x320 1 person, 70.7ms
Speed: 1.2ms preprocess, 70.7ms inference, 6.1ms postprocess per image at shape (1, 3, 256, 320)
Results saved to runs/detect/predict2


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'person', 1: 'weapon'}
 obb: None
 orig_img: array([[[120, 142, 160],
         [121, 143, 161],
         [122, 144, 162],
         ...,
         [133, 145, 151],
         [132, 144, 150],
         [131, 143, 149]],
 
        [[122, 144, 162],
         [123, 145, 163],
         [124, 146, 164],
         ...,
         [134, 146, 152],
         [132, 144, 150],
         [131, 143, 149]],
 
        [[122, 144, 162],
         [123, 145, 163],
         [126, 146, 164],
         ...,
         [135, 147, 153],
         [133, 145, 149],
         [132, 144, 148]],
 
        ...,
 
        [[ 91,  97, 102],
         [ 90,  96, 101],
         [ 90,  96, 101],
         ...,
         [ 35,  27,  28],
         [ 36,  28,  29],
         [ 37,  29,  30]],
 
        [[ 93,  99, 104],
         [ 92,  98, 103],
         [ 91,  97, 102],
         ...,
     

WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.
WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.
WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.
WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.
WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.
WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.
WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.
